In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tqdm import tqdm
import warnings
from keras.layers import *
from keras import Model

In [6]:
warnings.filterwarnings('ignore')
plt.rcParams["figure.dpi"] = 120
image_dimension = (224, 224)
image_input_size = (224, 224, 1)
epochs=100
batch_size = 32
epoch_index = [i for i in range(1, epochs+1)]

#### Load data

In [7]:
images = np.load(os.path.join("Brain_MRI_data1", "images.npy"), allow_pickle=True)
masks = np.load(os.path.join("Brain_MRI_data1", "masks.npy"), allow_pickle=True)

#### Change all image to grey

In [8]:
image_index = 0
for image in tqdm(images):
    plt.imsave(os.path.join("Brain_MRI_Segment_image", "image_{}.jpg".format(image_index)), image, cmap='gray')
    image_index += 1

mask_index = 0
for mask in tqdm(masks):
    plt.imsave(os.path.join("Brain_MRI_Segment_mask", "mask_{}.jpg".format(mask_index)), mask, cmap='gray')
    mask_index += 1

100%|██████████| 3064/3064 [00:16<00:00, 184.35it/s]


In [17]:
image_dir_name = "Brain_MRI_Segment_image"
mask_dir_name = "Brain_MRI_Segment_mask"
image_list = list()
mask_list = list()

images_dir = sorted(os.listdir(image_dir_name))
for image in tqdm(images_dir):
    current_image = cv2.imread(os.path.join(image_dir_name, image), cv2.IMREAD_GRAYSCALE)
    current_image = cv2.resize(current_image, image_dimension)
    image_list.append(current_image)


masks_dir = sorted(os.listdir(mask_dir_name))
for mask in tqdm(masks_dir):
    current_mask = cv2.imread(os.path.join(mask_dir_name, mask), cv2.IMREAD_GRAYSCALE)
    current_mask = cv2.resize(current_mask, image_dimension)
    mask_list.append(current_mask)

features = np.array(image_list)/255.0
masks = np.array(mask_list)/255.0
features = features.reshape((features.shape[0], features.shape[1], features.shape[2], 1))
masks = masks.reshape((masks.shape[0], masks.shape[1], masks.shape[2], 1))

100%|██████████| 3064/3064 [00:00<00:00, 4550.15it/s]


In [18]:
print("image dimension: {}, label dimension: {}".format(features.shape, masks.shape))

image dimension: (3064, 224, 224, 1), label dimension: (3064, 224, 224, 1)


#### Train, Test, Validation Split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features, masks, test_size=0.25, random_state=42, shuffle=True)
train_generator = ImageDataGenerator(validation_split=0.25)
test_generator = ImageDataGenerator()

train_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="training")
validation_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="validation", seed=42)
test_df = train_generator.flow(X_test, y_test, batch_size=batch_size)

#### U-Net Model Definition

In [20]:
inputs = Input(image_input_size)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
Unet = Model(inputs=[inputs], outputs=[outputs])

2023-02-18 00:12:09.828544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-18 00:12:09.830088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [21]:
Unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
Unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 224, 224, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [22]:
unet_history = Unet.fit(train_df,
                        epochs=epochs,
                        verbose=1,
                        validation_data=validation_df)

Epoch 1/100


2023-02-18 00:13:11.428013: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-18 00:13:12.517998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


54/54 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5536

2023-02-18 00:14:02.737365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


54/54 [==============================] - 58s 946ms/step - loss: 0.0000e+00 - accuracy: 0.5536 - val_loss: 0.0000e+00 - val_accuracy: 0.7526
Epoch 2/100
54/54 [==============================] - 48s 883ms/step - loss: 0.0000e+00 - accuracy: 0.7531 - val_loss: 0.0000e+00 - val_accuracy: 0.9133
Epoch 3/100
54/54 [==============================] - 47s 879ms/step - loss: 0.0000e+00 - accuracy: 0.9186 - val_loss: 0.0000e+00 - val_accuracy: 0.9755
Epoch 4/100
54/54 [==============================] - 44s 814ms/step - loss: 0.0000e+00 - accuracy: 0.9748 - val_loss: 0.0000e+00 - val_accuracy: 0.9777
Epoch 5/100
54/54 [==============================] - 44s 808ms/step - loss: 0.0000e+00 - accuracy: 0.9776 - val_loss: 0.0000e+00 - val_accuracy: 0.9799
Epoch 6/100
54/54 [==============================] - 45s 825ms/step - loss: 0.0000e+00 - accuracy: 0.9786 - val_loss: 0.0000e+00 - val_accuracy: 0.9799
Epoch 7/100
54/54 [==============================] - 43s 793ms/step - loss: 0.0000e+00 - accuracy: 0

KeyboardInterrupt: 